In [1]:
#pip install selenium
#pip install folium 
#pip install tejapi
import tejapi 
import folium.plugins
import pandas as pd
import os
from selenium import webdriver
#金鑰
tejapi.ApiConfig.api_key = "yourkey"

### Folium 地圖視覺化 範例

In [2]:
TPE_COORDINATES = (24.1075342,  120.6000887)
m = folium.Map(location=TPE_COORDINATES, zoom_start=8 , tiles="Stamen Terrain")
m

### 實價登錄資料搭配地圖視覺化

In [3]:
## 資料格式
data = tejapi.get("TWN/AAPRTRAN",coid="A",ann_date={"gt":"2020-12-01"},opts={"sort":"tot_prc.desc"},paginate=True)
data.tail(1)

,coid,mdate,key3,district_id,district,tran_tp,tran_tp_fg,tsign,tsign_fg,location,...,berth_tp_fg,berth_area,tot_prc,unit_prc,berth_prc,rmk,ann_date,abnorm_td,abnorm_loc,abnorm_prc
None,,,,,,,,,,,,,,,,,,,,,
4975,A,2020-12-23 00:00:00+00:00,RPQPMLLLKIPFFEA08DA,北投區,112,不動產,A,土地,A,立農段三小段481~510地號,...,,0.0,0.0,0.0,0.0,包含公共設施保留地用地;,2021-01-21 00:00:00+00:00,,,Y


In [4]:
### 資料點經緯度
data[["coord_x","coord_y"]]

,coord_x,coord_y
None,,
0,25.064356,121.580886
1,25.057765,121.611839
2,25.086624,121.564431
3,25.058210,121.574534
4,25.108892,121.526513
...,...,...
4971,25.023249,121.562360
4972,NaN,NaN
4973,25.040993,121.582211


In [5]:
if __name__ == "__main__": 
#   TEJ API 設定 
    data = tejapi.get("TWN/AAPRTRAN",coid="A",ann_date={"gt":"2020-12-01"},opts={"sort":"tot_prc.desc"},paginate=True)
    ##   本次資料區間用2020年12月之後，已有接近5000筆資料
    ##   資料比數如果太多，需要執行時間會較久
    
    ##   產生地圖TXT 設定基礎座標
    TPE_COORDINATES = (24.1075342,  120.6000887)
    m = folium.Map(location=TPE_COORDINATES, zoom_start=8 , tiles="Stamen Terrain")
    cluster = folium.plugins.MarkerCluster().add_to(m)
    
    ##  將實價登陸資料座標匯入folium當中
    for rowNumber in data.index.tolist():
        coordinate_X = None
        coordinate_Y = None
        tot_prc = None  
        location = None
        for columName in data.columns:
            val = data.get(columName)[rowNumber]
            #print(data.get(columName)[rowNumber])
            if pd.isnull(val):
                val = ""
            val = str(val)
            if columName=="coord_x":
                coordinate_X = val
            elif columName=="coord_y":
                coordinate_Y = val
            elif columName=="tot_prc":   
                tot_prc = val
            elif columName=="location":
                location = val
        if coordinate_X == "" or coordinate_Y == "":
            continue
#       print(str(rowNumber)+" # "+coordinate_X+","+coordinate_Y)
        folium.Marker(location=[float(coordinate_X), float(coordinate_Y)]     
                      ,popup =str(location +"<br/> 售價金額: " + tot_prc+"萬")
                      ,icon = folium.Icon(color="blue",icon="cloud")
                      ).add_to(cluster)              
        folium.LayerControl(collapsed=False).add_to(m) 
    m.save("地圖.html")
    
    ##  先透過 https://sites.google.com/a/chromium.org/chromedriver/home 下載Google chromedriver
    ##  再將html匯出到指定路徑中
    try:
        print(os.getcwd())
        path = os.getcwd() + "\\"
        driver = webdriver.Chrome(executable_path = path + "chromedriver.exe")
        driver.get(path + "地圖.html")
    except Exception as e:
        raise
    finally:
        driver.quit()
    pass

C:\Users\2021011902\Desktop\TEJAPI\TEJ_Github
